In [1]:
# All the neccessary imports
import os
import re
import io
import gc
import json
#imports required for reading docx files
import docx2txt
from io import StringIO
#imports required for reading text from PDF files
from PIL import Image
import pytesseract
from wand.image import Image as wi
#imports required for converting .doc to .docx file 
from win32com import client as wc
# imports required for reading docx files 
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

# importing pandas
import pandas as pd 

#initialization
doc_file_list = []
docx_file_list = []
pdf_file_list = []
resume_path = r'C:\Users\Param\Python-Project\Data\Resume'
phonePattern = r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'
emailPattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
github = r'(https?://github.com/[^\s]+)'
linkedin = r'(https?://www.linkedin.com/[^\s]+)'
emailMatches = {}
phoneNumerMatches = {}
skillsMatches = {}
githubMatches = {}
linkedinMatches = {}
allInOne = {}
emailList = []
phoneList = []
skillsList = []
githubList = []
linkedinList = []
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Opening PDF File using pdfminer
def pdf_resume_parser(fileName, path):
    print('reading file from -> ', path)
    output_string = StringIO()
    with open(path, 'rb') as pdf_file:
        parser = PDFParser(pdf_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    pdfText = output_string.getvalue()
    # this condition works when pdf is not searchable
    # or probably a image 
    if pdfText.isspace():
        pdf=wi(filename=path,resolution=300)                                                                                                                
        pdfImg=pdf.convert('jpeg')                                                                                                                                                                              
        imgBlobs=[]
        extracted_text=[]
        try:        
            for img in pdfImg.sequence:
                page=wi(image=img)
                imgBlobs.append(page.make_blob('jpeg'))
                for i in range(0,5):
                    [gc.collect() for i in range(0,10)]

            for imgBlob in imgBlobs:
                im=Image.open(io.BytesIO(imgBlob))
                text=pytesseract.image_to_string(im,lang='eng')
                text = text.replace(r"\n", " ")
                extracted_text.append(text)
                for i in range(0,5):
                    [gc.collect() for i in range(0,10)]
                pdfText = (''.join([i.replace("\n"," ").replace("\n\n"," ") for i in extracted_text]))
            [gc.collect() for i in range(0,10)]
        finally:
            [gc.collect() for i in range(0,10)]
            img.destroy()
       
    
    # check if mactches only then add to the respective list 
    emailList = [] 
    phoneList = [] 
    skillsList = [] 
    githubList = []
    linkedinList = []
    if len(re.findall(emailPattern,pdfText)) != 0:
        emailList.append(re.findall(emailPattern,pdfText))
    if len(re.findall(phonePattern,pdfText)) != 0:
        phoneList.append(re.findall(phonePattern,pdfText))
    if len(re.findall(github,pdfText)) != 0:
        githubList.append(re.findall(github,pdfText))
    if len(re.findall(linkedin,pdfText)) != 0:
        linkedinList.append(re.findall(linkedin,pdfText))    
    
    # check if skills found
    foundAt = pdfText.lower().find('skills') + 6
    clean_skills = pdfText[foundAt:].replace('\n','').replace('\t','').replace(r'\u','')
    skills = clean_skills.split(',')
    skillsList.append(skills[:-1])         
    # this condition will check if there is any email found or not, 
    # if not then marked it as 'NA'     
    if len(emailList) != 0:
        emailMatches[fileName] = emailList
    else:
        emailMatches[fileName] = 'NA'  
    # this condition will check if there is any phone Number found or not,
    # if not then marked it as 'NA'     
    if len(phoneList) != 0:
        phoneNumerMatches[fileName] = phoneList
    else:
        phoneNumerMatches[fileName] = 'NA'
    # this condition will check if there is any skills found or not,
    # if not then marked it as 'NA'
    if len(skillsList) != 0:
        skillsMatches[fileName] = skillsList
    else:
        skillsMatches[fileName] = 'NA' 
        
    if len(githubList) != 0:
        githubMatches[fileName] = githubList
    else:
        githubMatches[fileName] = "NA"    
        
    if len(linkedinList) != 0:
        linkedinMatches[fileName] = linkedinList
    else:
        linkedinMatches[fileName] = "NA"   
         
# reading docx file using docx2txt
def docx_resume_parser(fileName, path): 
    print('reading docx file from -> ', path)
    resume_docx = docx2txt.process(path)
    # check if mactches only then add to the respective list 
    # initialization of email and phonelist 
    emailList = [] 
    phoneList = []
    skillsList = []
    githubList = []
    linkedinList = []
    
    if len(re.findall(emailPattern,resume_docx)) != 0:
        emailList.append(re.findall(emailPattern,resume_docx))
    if len(re.findall(phonePattern,resume_docx)) != 0:
        phoneList.append(re.findall(phonePattern,resume_docx))
    if len(re.findall(github,resume_docx)) != 0:
        githubList.append(re.findall(github,resume_docx))
    if len(re.findall(linkedin,resume_docx)) != 0:
        linkedinList.append(re.findall(linkedin,resume_docx))
    # this condition will check if there is any email found or not, 
    # if not then marked it as 'NA'    
    if len(emailList) != 0:
        emailMatches[fileName] = emailList
    else:
        emailMatches[fileName] = 'NA'
    # this condition will check if there is any phone Number found or not, if not
    # then marked it as 'NA'     
    if len(phoneList) != 0:
        phoneNumerMatches[fileName] = phoneList
    else:
        phoneNumerMatches[fileName] = 'NA' 

    # finding skill set    
    foundAt = resume_docx.lower().find('skill') + 6
    clean_skills = resume_docx[foundAt:].replace('\n','').replace('\t','').replace(r'\u','')
    skills = clean_skills.split(',')
    skillsList.append(skills[:-1])
    if len(skillsList) != 0:
        skillsMatches[fileName] = skillsList
    else:
        skillsMatches[fileName] = 'NA' 
        
    if len(githubList) != 0:
        githubMatches[fileName] = githubList
    else:
        githubMatches[fileName] = "NA"    
        
    if len(linkedinList) != 0:
        linkedinMatches[fileName] = linkedinList
    else:
        linkedinMatches[fileName] = "NA"  
        
           
#reading doc file
def doc_resume_parser(fileName, filepath):
    print('reading doc file from -> ' + filepath)
    word = wc.Dispatch('Word.Application')
    doc = word.Documents.Open(os.path.abspath(filepath))
    # saving .doc file as .docx
    docx_file = filepath + 'x'
    doc.SaveAs(docx_file,16)
    doc.Close()
    word.Quit()

    # checking if file got saves successfully or not, if yes then reading again
    #as docx and once reading finished then removing docx file from path
    if os.path.exists(docx_file):
        docx_resume_parser(fileName,docx_file)
        os.remove(docx_file)
      
    
#reading all types of files from local system
files = os.listdir(resume_path)
for file in files:
    if file.endswith('.doc'):
        doc_file_list.append(file)
    elif file.endswith('.docx'):
        docx_file_list.append(file)
    elif file.endswith('.pdf'):
        pdf_file_list.append(file)


In [2]:
# parsing all docx format resume for email pattern
for docx in docx_file_list:
    path = resume_path + '\\' + docx
    docx_resume_parser(docx, path)


# parsing all doc format resume for email pattern
for doc in doc_file_list:
    path = resume_path + r'\\' + doc
    doc_resume_parser(doc, path)


#parsing all pdf format resume for email pattern
for pdf in pdf_file_list:
    path = resume_path + '\\' + pdf
    pdf_resume_parser(pdf, path)


# Generating a common dict for email,phone and skills for all files
for match in emailMatches.keys():
    if match in phoneNumerMatches:
        if match in skillsMatches:
            if match in githubMatches:
                if match in linkedinMatches:               
                    allInOne[match] = { 'Resume' : match,
                                        'Email'  : emailMatches[match],
                                        'Phone'  : phoneNumerMatches[match],
                                        'GitHub' : githubMatches[match],
                                        'LinkedIn' : linkedinMatches[match],
                                        'Skills' : skillsMatches[match]}

reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\12+ (2).docx
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\12+.docx
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\3+ (2).docx
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\3+.docx
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\4+.docx
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\8+.docx
reading doc file from -> C:\Users\Param\Python-Project\Data\Resume\\4.doc
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\\4.docx
reading doc file from -> C:\Users\Param\Python-Project\Data\Resume\\freasher 1.doc
reading docx file from ->  C:\Users\Param\Python-Project\Data\Resume\\freasher 1.docx
reading file from ->  C:\Users\Param\Python-Project\Data\Resume\15+ (1).pdf
reading file from ->  C:\Users\Param\Python-Project\Data\Resume\15+.pdf
reading file from ->  C:\Users\Param\Python-Project\Data\Res

In [3]:
df = pd.DataFrame.from_dict(allInOne).transpose()
df.set_index('Resume', inplace = True)

In [8]:
df.to_csv("Resume_Parser.csv")